In [13]:
import chipwhisperer as cw
import struct

In [25]:
def float_to_binary_str(f):
    # Pack the float into 4 bytes (32-bit) using IEEE 754 standard
    [packed] = struct.unpack('!I', struct.pack('!f', f))
    # Convert the packed number to a binary string
    return f"{packed:032b}"

In [26]:
def HW_float32(f):
    # Get the binary representation of the 32-bit float
    binary_str = float_to_binary_str(f)
    # Count and return the number of '1' bits
    return binary_str.count('1')

In [28]:
print(float_to_binary_str(0.657))
print(HW_float32(0.657))

00111111001010000011000100100111
15


In [2]:
proj = cw.open_project('project-02')

In [3]:
trace_waves_arr = []
for trace in proj.traces:
    trace_waves_arr.append(trace.wave)

In [4]:
print(len(trace_waves_arr))

1000


In [93]:
def CPA_attack(trace_waves, inputs, weights, time_sample_index):
    r_abs = []
    n_trace_waves = len(trace_waves)
    leakage_sum = 0
    for trace_wave in trace_waves:
        leakage_sum += trace_wave[time_sample_index]    
    average_leakage = leakage_sum / n_trace_waves
    M = len(inputs)
    N = len(weights)
    
    
    for i in range(N):
        hypothetical_products = []
        hypothetical_leakages = []
        hypothetical_leakage_sum = 0
        for j in range(M):
            hypothetical_product = weights[i] * inputs[j]
            hypothetical_products.append(hypothetical_product)
            hypothetical_leakage = HW_float32(hypothetical_product)
            hypothetical_leakages.append(hypothetical_leakage)
            hypothetical_leakage_sum += hypothetical_leakage
            #print(f'Input {inputs[j]} with idx {j}, weight {weights[i]} with idx {i} -> hypothetical_product = {hypothetical_product}')
            #print(hypothetical_leakage)
        average_hypothetical_leakage = hypothetical_leakage_sum / M
        #print(average_hypothetical_leakage)
        
        
        numerator= 0
        denominator1 = 0
        denominator2 = 0
        for j in range(M):
            numerator += (hypothetical_leakages[j] - average_hypothetical_leakage) * (trace_waves[j][time_sample_index] - average_leakage)
            denominator1 += (hypothetical_leakages[j] - average_hypothetical_leakage)**2
            denominator2 += (trace_waves[j][time_sample_index] - average_leakage)**2
            #print(f'Input {inputs[j]} with idx {j}, weight {weights[i]} with idx {i} -> hypothetical_product = {hypothetical_products[j]}' +
            #     f' hypothetical_leakage = {hypothetical_leakages[j]}')
        #print(numerator, denominator1, denominator2)
        denominator1 = denominator1**(1/2)
        denominator2 = denominator2**(1/2)
        #print(numerator, denominator1, denominator2)
        
        corr_coef = numerator / (denominator1 * denominator2)
        r_abs.append(abs(corr_coef))
        #print(abs(corr_coef))
    return r_abs
        
            
            
            

In [102]:

test_inputs = [0.5, 0.657, 0.3333, 0.0003, 0.189, 0.5243, 0.2222, 0.4959]
test_trace_waves = trace_waves_arr[:8]
test_weights = [0.21, 0.22, 0.23]
test_time_sample_index = 5

In [103]:
CPA_attack(trace_waves = test_trace_waves,
           inputs = test_inputs,
           weights = test_weights,
           time_sample_index = test_time_sample_index)

[0.32816263481641533, 0.14998263829050926, 0.10630051661265462]